In [1]:
# Import Libraries
from ta import add_all_ta_features
import pandas as pd
import requests
import io
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time

C:\Users\imaia\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Import main price dataset from Github

# Downloading the csv file from your GitHub account
url = "https://raw.githubusercontent.com/Yu821/stock_price_prediction/main/data/price.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
financial = pd.read_csv(io.StringIO(download.decode('utf-8')))


In [3]:
financial.iloc[:,0:20].head()

,change,changeOverTime,changePercent,close,fClose,fHigh,fLow,fOpen,fVolume,high,id,key,label,low,marketChangeOverTime,open,subkey,symbol,uClose,uHigh
0,0.00,0.000000,0.0000,123.00,123.00,124.1800,122.4900,123.66,75089134,124.1800,HISTORICAL_PRICES,AAPL,"Apr 1, 21",122.4900,0.000000,123.66,NaN,AAPL,123.00,124.1800
1,-0.85,-0.006911,-0.0069,122.15,122.15,123.5200,121.1500,121.65,118323826,123.5200,HISTORICAL_PRICES,AAPL,"Mar 31, 21",121.1500,-0.006911,121.65,NaN,AAPL,122.15,123.5200
2,-2.25,-0.025203,-0.0184,119.90,119.90,120.4031,118.8600,120.11,85671919,120.4031,HISTORICAL_PRICES,AAPL,"Mar 30, 21",118.8600,-0.025203,120.11,NaN,AAPL,119.90,120.4031
3,1.49,-0.013089,0.0124,121.39,121.39,122.5800,120.7299,121.65,80819203,122.5800,HISTORICAL_PRICES,AAPL,"Mar 29, 21",120.7299,-0.013089,121.65,NaN,AAPL,121.39,122.5800
4,-0.18,-0.014553,-0.0015,121.21,121.21,121.4800,118.9200,120.35,94071234,121.4800,HISTORICAL_PRICES,AAPL,"Mar 26, 21",118.9200,-0.014553,120.35,NaN,AAPL,121.21,121.4800


In [4]:
# Clean financial dataset

financial['date'] = pd.DatetimeIndex(financial['label'])

financial['date'] = financial['date'].dt.strftime('%Y-%m-%d')

financial.shape

(12588, 26)

In [5]:
finance = financial.loc[:,['date', "open", "high", "low", "close", "volume", 'symbol',
                     "marketChangeOverTime", 'changeOverTime', "changePercent"]]

finance.date = pd.DatetimeIndex(finance['date'])

finance.date = finance['date'].dt.strftime('%Y-%m-%d')

print(finance.shape)
finance.head()

(12588, 10)


,date,open,high,low,close,volume,symbol,marketChangeOverTime,changeOverTime,changePercent
0,2021-04-01,123.66,124.1800,122.4900,123.00,75089134,AAPL,0.000000,0.000000,0.0000
1,2021-03-31,121.65,123.5200,121.1500,122.15,118323826,AAPL,-0.006911,-0.006911,-0.0069
2,2021-03-30,120.11,120.4031,118.8600,119.90,85671919,AAPL,-0.025203,-0.025203,-0.0184
3,2021-03-29,121.65,122.5800,120.7299,121.39,80819203,AAPL,-0.013089,-0.013089,0.0124
4,2021-03-26,120.35,121.4800,118.9200,121.21,94071234,AAPL,-0.014553,-0.014553,-0.0015


In [6]:
stock_news = pd.read_csv (r'C:\Users\imaia\Desktop\Stats170A\Capstone\stock_news_data.csv')

stock_news = stock_news.groupby('date').agg({'score' : 'mean'}).reset_index()

market_news = pd.read_csv (r'C:\Users\imaia\Desktop\Stats170A\Capstone\market_news_data.csv')

market_news = market_news.groupby('date').agg({'score' : 'mean'}).reset_index()

news_scores = stock_news.merge(market_news, on = 'date')

news_scores.columns = ["date", "stock_news_score", "market_news_score"]

print(news_scores.shape)
news_scores.head()

(560, 3)


,date,stock_news_score,market_news_score
0,2019-04-02,-0.140500,-0.458800
1,2019-04-03,0.190633,-0.431067
2,2019-04-04,-0.004880,0.014725
3,2019-04-06,0.120400,0.177900
4,2019-04-08,0.251600,0.102700


In [7]:
finance = finance.merge(news_scores, on = 'date', how = 'left')
finance = finance.fillna(0)
finance.tail()

,date,open,high,low,close,volume,symbol,marketChangeOverTime,changeOverTime,changePercent,stock_news_score,market_news_score
12583,2016-04-08,54.67,55.28,54.32,54.42,22167223,MSFT,-0.775449,-0.775449,0.0020,0.0,0.0
12584,2016-04-07,54.87,54.91,54.23,54.46,19225092,MSFT,-0.775284,-0.775284,0.0007,0.0,0.0
12585,2016-04-06,54.36,55.20,54.21,55.12,21188682,MSFT,-0.772560,-0.772560,0.0121,0.0,0.0
12586,2016-04-05,55.19,55.30,54.46,54.56,19272255,MSFT,-0.774871,-0.774871,-0.0102,0.0,0.0
12587,2016-04-04,55.43,55.66,55.00,55.43,18928810,MSFT,-0.771281,-0.771281,0.0159,0.0,0.0


In [8]:
technical = add_all_ta_features(finance, open="open", high="high", low="low", close="close",
                                     volume="volume", fillna=True)

print(technical.shape)
technical = technical.sort_values(by='date')

technical.head()

C:\Users\imaia\Anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\imaia\Anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


(12588, 95)


,date,open,high,low,close,volume,symbol,marketChangeOverTime,changeOverTime,changePercent,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
6293,2016-04-04,39.36,39.3800,38.41,38.79,5381340,PYPL,-0.843298,-0.843298,0.0062,...,-54.828660,-0.587610,38.742785,-3.218563,-9.351761,-7.508740,-1.843020,0.622568,0.620638,-68.463415
11329,2016-04-04,2.83,2.8700,2.80,2.83,5591055,AMD,-0.965101,-0.965101,0.0254,...,-14.814815,-0.520456,3.105416,4.814815,-25.472047,-15.787949,-9.684098,2.536232,2.504603,-97.699187
10070,2016-04-04,29.02,29.9889,28.75,29.30,1619759,SHOP,-0.974641,-0.974641,0.0191,...,-82.400000,-0.109333,29.654166,-6.984127,8.399905,-7.877988,16.277893,1.913043,1.894975,-76.178862
8811,2016-04-04,50.81,51.1110,50.51,50.65,6289819,QCOM,-0.632412,-0.632412,0.0116,...,-67.140320,-1.037221,51.269756,-2.764446,-8.151087,-3.759601,-4.391486,1.158378,1.151720,-58.821138
7552,2016-04-04,78.33,79.5500,78.32,79.08,9817315,BABA,-0.647531,-0.647531,0.0228,...,-52.335724,-0.538865,78.668154,-2.622830,-11.310831,-12.013157,0.702326,2.276255,2.250734,-35.707317


In [9]:
technical = technical.dropna(axis=1)
technical.shape

(12588, 95)

In [10]:
cop = technical.copy()

Y = pd.DataFrame(technical[['open', 'date', 'symbol']])

X = pd.DataFrame(technical)

print(X.shape)

(12588, 95)


In [50]:
from sklearn.model_selection import train_test_split

def reshape_data(X, Y):
    # split into samples
    X_samples = list()
    y_samples = list()

    NumerOfRows = X.shape[0]

    TimeSteps = 7 # next day's Price Prediction is based on last how many past day's prices
    FutureTimeSteps = 1 # How many days in future you want to predict the prices

    # Iterate thru the values to create combinations
    for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
        x_sample = X[i-TimeSteps:i]
        y_sample = Y[i:i+FutureTimeSteps]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

    # Reshape the Input as a 3D (number of samples, Time Steps, Features)
    X_data = np.array(X_samples)
    X_data = X_data.reshape(X_data.shape[0], X_data.shape[1], X.shape[1])

    # We do not reshape y as a 3D data  as it is supposed to be a single column only
    y_data = np.array(y_samples)
    y_data = y_data.reshape(y_data.shape[0], FutureTimeSteps)
    
    return X_data, y_data

def split_data(X, Y):
    X_train = X[:int(X.shape[0]*0.90)]
    X_test = X[int(X.shape[0]*0.90):]
    y_train =  Y[:int(Y.shape[0]*0.90)]
    y_test =  Y[int(Y.shape[0]*0.90):]
    
    return X_train, X_test, y_train, y_test


def calculate_accuracy(X_test, y_test, regressor):
    
    y_pred = DataScaler_Y.inverse_transform(regressor.predict(X_test))
    Y_Test = DataScaler_Y.inverse_transform(y_test)

    percent = 1

    total = len(Y_Test)

    correct = 0

    for i in range(len(Y_Test)):
        upper = Y_Test[i] + (( percent / 100) * Y_Test[i] )
        lower = Y_Test[i] - (( percent / 100) * Y_Test[i] )
        if y_pred[i] < upper and y_pred[i] > lower:
            correct = correct + 1
        else:
            pass
        
    return (correct/total)*100


# Feature Scaling for fast training of neural networks
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def normalize(X, Y):

    sc = MinMaxScaler()

    DataScaler_X = sc.fit(X)
    transformed_X = DataScaler_X.transform(X)
    X = pd.DataFrame(transformed_X, columns = X.columns)

    DataScaler_Y = sc.fit(Y)
    transformed_Y = DataScaler_Y.transform(Y)
    Y = pd.DataFrame(transformed_Y, columns = Y.columns)
    
    return X, Y, DataScaler_Y


def remove_uncorrelated(X):
    # Find the attributes that have a correlated of greater than 0.5
    correlated = X.corr()
    
    p_indices = np.where(correlated > 0.1)
    p_indices = [(correlated.index[x], correlated.columns[y]) for x, y in zip(*p_indices)
                                            if x != y and x < y]

    n_indices = np.where(correlated < -0.1)
    n_indices = [(correlated.index[x], correlated.columns[y]) for x, y in zip(*n_indices)
                                            if x != y and x < y]

    significant_indicators = set()

    # find the attributes that are directly correlated to the opening price
    for i in p_indices:
        if i[0] == 'open':
            significant_indicators.add(i[1])
        else:
            pass

    for i in n_indices:
        if i[0] == 'open':
            significant_indicators.add(i[1])
        else:
            pass

    significant_indicators = list(significant_indicators)
    
    return X[significant_indicators]



In [51]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Masking, TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras import optimizers

In [57]:
# Parameters to try going through
hidden_1 = [15, 35, 65]
hidden_2 = [10, 15, 30]
batches = [20, 50, 100]
epoches = [65, 110, 135]
lrs = [0.01, 0.001, 0.0001]


In [58]:
def find_optimal_model(X_train, X_test, y_train, y_test, DataScaler_Y):
    
    best_model = None
    best_accuracy = 100
    TimeSteps = X_train.shape[1]
    TotalFeatures = X_train.shape[2]
    
    for cell in range(len(hidden_1)):
            for batch in batches:
                for epoch in epoches:
                    for lr in lrs:

                        # Initialising the RNN
                        regressor = Sequential()

                        regressor.add(LSTM(units = 12, activation = 'relu', input_shape = (TimeSteps, TotalFeatures)))#, return_sequences=True))

                        regressor.add(Dense(8))

                        regressor.add(Dense(1, kernel_initializer = 'uniform', activation='linear'))

                        # Compiling the RNN
                        optimizer = optimizers.Adam(lr=lr)
                        regressor.compile(optimizer = optimizer, loss = 'mean_squared_error')

                        # Fitting the RNN to the Training set
                        regressor.fit(X_train, y_train, batch_size = batch, epochs = epoch, verbose = 0)

                        accuracy = calculate_accuracy(X_test, y_test, regressor)
                        print(accuracy)

                        if best_accuracy > accuracy:
                            best_accuracy = accuracy 
                            best_model = regressor
                    
    return best_model

                    

In [59]:
# Stocks we will build models for 
symbols= ["FB"] #, "FB", "ADBE", "INTC", "PYPL", "BABA", "QCOM", "SHOP", "AMD", "MSFT"]

In [60]:
all_models = []

for i in symbols:
    
    X_sym = X[X['symbol'] == i]
    Y_sym = Y[Y['symbol'] == i]
    
    X_sym = X_sym.drop(['date', 'symbol'], axis = 1)
    Y_sym = Y_sym.drop(['date', 'symbol'], axis = 1)
    
    X_sym = remove_uncorrelated(X_sym)
    
    X_sym, Y_sym, DataScaler_Y = normalize(X_sym, Y_sym)
    
    X_sym, Y_sym = reshape_data(X_sym, Y_sym)
    
    X_train, X_test, y_train, y_test = split_data(X_sym, Y_sym)
    
    models = find_optimal_model(X_train, X_test, y_train, y_test, DataScaler_Y)
    
    all_models = all_models.append((i, models))
    

38.095238095238095
38.095238095238095
27.77777777777778
50.0
43.65079365079365
21.428571428571427
19.047619047619047
52.38095238095239
32.53968253968254
42.857142857142854
53.17460317460318
26.190476190476193
29.365079365079367
42.06349206349206
16.666666666666664
40.476190476190474
44.44444444444444
21.428571428571427
26.984126984126984
36.507936507936506
18.253968253968253
6.349206349206349
33.33333333333333
26.190476190476193
48.41269841269841
24.6031746031746
30.158730158730158
19.047619047619047
53.96825396825397
18.253968253968253
38.095238095238095
46.82539682539682
39.682539682539684
43.65079365079365
57.14285714285714
28.57142857142857
7.142857142857142
35.714285714285715
25.396825396825395
42.857142857142854
48.41269841269841
25.396825396825395
47.61904761904761
50.0
38.88888888888889
57.936507936507944
33.33333333333333
19.841269841269842
46.03174603174603
36.507936507936506
30.158730158730158
33.33333333333333
45.23809523809524
30.952380952380953
45.23809523809524
33.333333

In [61]:
all_models[0]

TypeError: 'NoneType' object is not subscriptable